In [2]:
import numpy 
import cv2 as cv
import os

In [3]:
if os.path.exists("data_folder") is not True: os.mkdir("data_folder")

video_path = r"C:\Users\524ha\Desktop\Araştırmalarım\data_managment\test.mp4"
cap = cv.VideoCapture(video_path)
count = -1
while True:
    count += 1
    ret, frame = cap.read()

    if ret is False:
        print("Frame is None returned")
        break
    if count % 3 == 0:
        basename = f".\\data_folder\\0508_{int(count/3)}.jpg" 

        cv.imwrite(basename,frame)
        print("yazildi", basename)

cap.release()

yazildi .\data_folder\0508_0.jpg
yazildi .\data_folder\0508_1.jpg
yazildi .\data_folder\0508_2.jpg
yazildi .\data_folder\0508_3.jpg
yazildi .\data_folder\0508_4.jpg
yazildi .\data_folder\0508_5.jpg
yazildi .\data_folder\0508_6.jpg
yazildi .\data_folder\0508_7.jpg
yazildi .\data_folder\0508_8.jpg
yazildi .\data_folder\0508_9.jpg
yazildi .\data_folder\0508_10.jpg
yazildi .\data_folder\0508_11.jpg
yazildi .\data_folder\0508_12.jpg
yazildi .\data_folder\0508_13.jpg
yazildi .\data_folder\0508_14.jpg
yazildi .\data_folder\0508_15.jpg
yazildi .\data_folder\0508_16.jpg
yazildi .\data_folder\0508_17.jpg
yazildi .\data_folder\0508_18.jpg
yazildi .\data_folder\0508_19.jpg
yazildi .\data_folder\0508_20.jpg
yazildi .\data_folder\0508_21.jpg
yazildi .\data_folder\0508_22.jpg
yazildi .\data_folder\0508_23.jpg
yazildi .\data_folder\0508_24.jpg
yazildi .\data_folder\0508_25.jpg
yazildi .\data_folder\0508_26.jpg
yazildi .\data_folder\0508_27.jpg
yazildi .\data_folder\0508_28.jpg
yazildi .\data_folder\05

In [2]:
import glob
import cv2
import os
from ultralytics import YOLO
import numpy as np
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class ObjectDetector:
    def __init__(self, model_path, conf_threshold=0.05):
        """
        Initialize the detector with model path and confidence threshold
        """
        self.model = YOLO(model_path)
        self.conf_threshold = conf_threshold
        
    def process_video(self, video_path, output_folder, target_classes=None):
        """
        Process video and save frames with detections.  
        Yeni sürüm: Model herhangi bir nesne tespit etmese bile her 4. kare görüntü ve etiketi (boş dosya) olarak kaydedilir.
        """
        # Create output folders
        frame_folder = os.path.join(output_folder, "images")
        annotation_folder = os.path.join(output_folder, "labels")
        annotated_frame_folder = os.path.join(output_folder, "annotateds")

        os.makedirs(frame_folder, exist_ok=True)
        os.makedirs(annotation_folder, exist_ok=True)
        os.makedirs(annotated_frame_folder, exist_ok=True)
        
        # Open video
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            raise ValueError(f"Error opening video file: {video_path}")
            
        frame_count = 0
        total_detections = 0
        
        try:
            i = 0
            while cap.isOpened():
                success, frame = cap.read()
                if not success:
                    break
                    
                i += 1

                frame_count += 1

                if i % 3 != 0: 
                    continue
                
                logger.info(f"Processing frame {frame_count}")
                
                # Run detection
                results = self.model.predict(
                    frame,
                    conf=self.conf_threshold,
                    iou=0.1,
                    classes=target_classes,
                    verbose=False
                )

                # Yalın kareyi her durumda kaydet
                frame_path = os.path.join(frame_folder, f"0508_{frame_count:06d}.jpg")
                cv2.imwrite(frame_path, frame)

                # İlgili etiket dosya yolu
                annotation_path = os.path.join(annotation_folder, f"0508_{frame_count:06d}.txt")

                # İşlenen her sonuç için (genelde tek bir sonuç döner)
                has_detections = False
                for result in results:
                    num_detections = len(result.boxes)
                    total_detections += num_detections

                    if num_detections > 0:
                        has_detections = True
                        # Etiket dosyasını kaydet
                        self._save_annotations(
                            result.boxes,
                            frame.shape,
                            annotation_path
                        )

                        # Log detection details
                        classes = result.boxes.cls.cpu().numpy()
                        confidences = result.boxes.conf.cpu().numpy()
                        logger.info(f"Frame {frame_count}: Found {num_detections} objects")
                        logger.info(f"Classes detected: {classes}")
                        logger.info(f"Confidence scores: {confidences}")

                        # Draw boxes on frame (optional)
                        annotated_frame = self._draw_boxes(frame, result.boxes)
                        cv2.imwrite(
                            os.path.join(
                                annotated_frame_folder,
                                f"0508_{frame_count:06d}_annotated.jpg"
                            ),
                            annotated_frame,
                        )

                # Eğer hiçbir nesne tespit edilmediyse, boş etiket dosyası oluştur
                if not has_detections:
                    open(annotation_path, "w").close()
                    logger.info(f"Frame {frame_count}: No detections (empty label saved)")
                
        finally:
            cap.release()
            
        logger.info(f"Processing complete. Total frames: {frame_count}")
        logger.info(f"Total detections: {total_detections}")
        return frame_count, total_detections
    
    def _save_annotations(self, boxes, image_shape, output_path):
        """
        Save detections in YOLO format
        """
        img_height, img_width = image_shape[:2]
        
        with open(output_path, 'w') as f:
            for box in boxes:
                # Get box coordinates
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
                
                # Convert to YOLO format
                width = x2 - x1
                height = y2 - y1
                center_x = x1 + width / 2
                center_y = y1 + height / 2
                
                # Normalize coordinates
                center_x /= img_width
                center_y /= img_height
                width /= img_width
                height /= img_height
                
                # Write to file
                class_id = int(box.cls.item())
                f.write(f"{class_id} {center_x:.6f} {center_y:.6f} {width:.6f} {height:.6f}\n")
    
    def _draw_boxes(self, frame, boxes):
        """
        Draw detection boxes on frame
        """
        annotated_frame = frame.copy()
        
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
            conf = float(box.conf)
            class_id = int(box.cls.item())
            
            # Draw box
            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            
            # Add label
            label = f"Class {class_id}: {conf:.2f}"
            cv2.putText(
                annotated_frame,
                label,
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0, 255, 0),
                2,
            )
            
        return annotated_frame

def main():
    # Configuration
    MODEL_PATH = r"C:\Users\524ha\Desktop\Resources\data_managment\pt\best.pt"
    VIDEO_FILES = [
        r"C:\Users\524ha\Desktop\Resources\data_managment\unlabeled_1.mp4",
    ]
    # Uncomment below line to use glob for video files
    # VIDEO_FILES = glob.glob(r"*.mp4")
    print(VIDEO_FILES)
    OUTPUT_BASE = "2010"
    CONFIDENCE_THRESHOLD = 0.6
    TARGET_CLASSES = None  # Set to None to detect all classes, or specify list like [0, 1, 2]
    
    # Initialize detector
    detector = ObjectDetector(MODEL_PATH, CONFIDENCE_THRESHOLD)

    # Process each video
    for video_file in VIDEO_FILES:
        logger.info(f"Processing video: {video_file}")
        
        # Create output folder for this video
        video_name = os.path.splitext(os.path.basename(video_file))[0]
        output_folder = os.path.join(OUTPUT_BASE, video_name)
        os.makedirs(output_folder, exist_ok=True)
        
        # Process video
        try:
            frames, detections = detector.process_video(
                video_file,
                output_folder,
                target_classes=TARGET_CLASSES,
            )
            logger.info(f"Completed processing {video_file}")
            logger.info(f"Processed {frames} frames with {detections} total detections")
            
        except Exception as e:
            logger.error(f"Error processing {video_file}: {str(e)}")

if __name__ == "__main__":
    main()


['C:\\Users\\524ha\\Desktop\\Resources\\data_managment\\unlabeled_1.mp4']


INFO:__main__:Processing video: C:\Users\524ha\Desktop\Resources\data_managment\unlabeled_1.mp4
INFO:__main__:Processing frame 3
INFO:__main__:Frame 3: Found 1 objects
INFO:__main__:Classes detected: [          5]
INFO:__main__:Confidence scores: [    0.78767]
INFO:__main__:Processing frame 6
INFO:__main__:Frame 6: Found 1 objects
INFO:__main__:Classes detected: [          5]
INFO:__main__:Confidence scores: [    0.81579]
INFO:__main__:Processing frame 9
INFO:__main__:Frame 9: Found 1 objects
INFO:__main__:Classes detected: [          5]
INFO:__main__:Confidence scores: [    0.90584]
INFO:__main__:Processing frame 12
INFO:__main__:Frame 12: Found 1 objects
INFO:__main__:Classes detected: [          5]
INFO:__main__:Confidence scores: [    0.90343]
INFO:__main__:Processing frame 15
INFO:__main__:Frame 15: Found 1 objects
INFO:__main__:Classes detected: [          5]
INFO:__main__:Confidence scores: [    0.91157]
INFO:__main__:Processing frame 18
INFO:__main__:Frame 18: Found 1 objects
I

In [9]:
import os

path = r"C:/Users/524ha/Desktop/data_managment/minivalf_aygaz_Hasan"
print(os.path.exists(path))  # True olmalı
print(os.listdir(path))  # Klasördeki dosyaları listele


True
['minivalf_aygaz_Hasan']


In [3]:
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO

model = YOLO("best.pt")
cap = cv2.VideoCapture("mini_bos_video.mp4")
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    results = model.predict(frame)
    frame_ = results[0].plot()
    cv2.imshow("frame", frame_)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()



0: 384x640 (no detections), 105.9ms
Speed: 5.8ms preprocess, 105.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 45.2ms
Speed: 0.0ms preprocess, 45.2ms inference, 8.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 46.1ms
Speed: 4.7ms preprocess, 46.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 52.8ms
Speed: 0.0ms preprocess, 52.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 42.7ms
Speed: 0.0ms preprocess, 42.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 38.1ms
Speed: 6.7ms preprocess, 38.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 41.6ms
Speed: 5.0ms preprocess, 41.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 54.1ms
Speed: 0.0ms preprocess, 54.1ms

In [ ]:
from sahi import AutoDetectionModel
from sahi.utils.file import download_from_url
from sahi.utils.ultralytics import download_yolo11n_model
from sahi.predict import get_prediction


# Download YOLO11 model
model_path = "pt/kule_m_model_best.pt"
download_yolo11n_model(model_path)

detection_model = AutoDetectionModel.from_pretrained(
    model_type="ultralytics",
    model_path=model_path,
    confidence_threshold=0.3,
    device="cpu",  # or 'cuda:0'
)



FileNotFoundError: No such file: 'c:\Users\524ha\Desktop\AYGAZ_DATAS\data_managment\demo_data\small-vehicles1.jpeg'

In [2]:
import os
import cv2
import glob
from ultralytics import YOLO
from sahi import AutoDetectionModel
from sahi.predict import get_prediction

# YOLO modelini SAHI için yükleme
model_path = "pt/yandan.pt"
detection_model = AutoDetectionModel.from_pretrained(
    model_type="ultralytics",
    model_path=model_path,
    confidence_threshold=0.6,
    device="cuda",  # 'cuda:0' kullanabilirsin
)

def yolo_format(x_center, y_center, width, height, img_width, img_height):
    """ Normalize koordinatları YOLO formatına dönüştür """
    return f"{x_center / img_width} {y_center / img_height} {width / img_width} {height / img_height}"

def detect_and_label(image_path, output_dir):
    """ Resmi işler, nesneleri tespit eder ve YOLO formatında kaydeder """
    img = cv2.imread(image_path)
    height, width, _ = img.shape
    
    # SAHI ile tahmin al
    results = get_prediction(image_path, detection_model)
    
    labels_path = os.path.join(output_dir, os.path.basename(image_path).replace(".jpg", ".txt"))
    
    with open(labels_path, "w") as f:
        for obj in results.object_prediction_list:  # Doğrudan sonuçları kullan
            x1, y1, x2, y2 = obj.bbox.to_xyxy()  # SAHI bbox erişimi
            x_center = (x1 + x2) / 2
            y_center = (y1 + y2) / 2
            w = x2 - x1
            h = y2 - y1
            class_id = obj.category.id  # SAHI sınıf ID'si
            
            f.write(f"{class_id} {yolo_format(x_center, y_center, w, h, width, height)}\n")
    
    print(f"Etiket kaydedildi: {labels_path}")

def process_images(image_dir, output_dir):
    """ Dizindeki tüm resimleri YOLO formatında etiketle """
    os.makedirs(output_dir, exist_ok=True)
    images = glob.glob(os.path.join(image_dir, "*.jpg"))
    
    for image_path in images:
        detect_and_label(image_path, output_dir)
    
    print("Tüm resimler işlendi!")

# Kullanım
dataset_path = r"C:\Users\524ha\Desktop\AYGAZ_DATAS\Datasets\yarımca_kamyon\Annotation\images"  # Resimlerin olduğu klasör
output_labels = r"C:\Users\524ha\Desktop\AYGAZ_DATAS\Datasets\yarımca_kamyon\Annotation\images\labels"  # Etiketlerin kaydedileceği klasör

process_images(dataset_path, output_labels)


ModuleNotFoundError: No module named 'sahi'